In [1]:
import pandas as pd
from functools import reduce

In [2]:
data1 = pd.read_excel('整理後資料/購買意圖分數_直接購買.xlsx')
data2 = pd.read_excel('整理後資料/購買意圖分數_影響最深.xlsx')
data3 = pd.read_csv('整理後資料/消費金額_直接購買.csv', encoding='big5')
data4 = pd.read_csv('整理後資料/消費金額_影響最深.csv', encoding='big5')
data5 = pd.read_csv('整理後資料/廣告活動來源_影響最深.csv')
data6 = pd.read_csv('整理後資料/廣告活動媒介_影響最深.csv')
data7 = pd.read_csv('整理後資料/直接購買會員貼標資料.csv', encoding='utf8')
data8 = pd.read_csv('整理後資料/影響最深貼標資料（重複ID只留第一筆）.csv', encoding='utf8')

In [3]:
print(len(data1), len(data2), len(data3), len(data4), len(data5), len(data6), len(data7), len(data8))

1571 1959 1591 1962 1781 1715 1845 1958


In [4]:
# data7['month_usage_score'].unique()

In [5]:
direct = pd.read_csv('直接購買_uid.csv', encoding='big5')
ad = pd.read_csv('影響最深_uid.csv', encoding='big5')
direct_uids = list(direct['行為資料 uid'])
ad_uids = list(ad['行為資料 uid'])
direct_app_uids = list(direct['APP uid'])
ad_app_uids = list(ad['APP uid'])

In [6]:
print(len(data1.loc[data1['uid'].isin(direct_uids)]), len(data3.loc[data3['uid'].isin(direct_uids)]))
print(len(data2.loc[data2['uid'].isin(ad_uids)]), len(data4.loc[data4['uid'].isin(ad_uids)]), len(data5.loc[data5['uid'].isin(ad_uids)]), len(data6.loc[data6['uid'].isin(ad_uids)]))
print(len(data7.loc[data7['MemberID'].isin(direct_app_uids)]), len(data8.loc[data8['MemberID'].isin(ad_app_uids)]))

1569 1587
1958 1958 1772 1706
1845 1958


In [7]:
len(data7.loc[data7['MemberID'].isin(direct_app_uids)]['MemberID'])

1845

In [8]:
len(data8.loc[data8['MemberID'].isin(ad_app_uids)]['MemberID'].unique())

1958

In [9]:
data7 = data7.rename(columns={'location _highest': 'location_highest'})

In [10]:
data4

,uid,累計金額,每次平均消費金額
0,%2B%2B6rOXDBdSgRq7Ks8WEtjF%2FacB6jaVRXpqWA4HRN...,1000~2000,1000~2000
1,%2B3ewi8VLqI8BqfOC%2F4jmUyOwhunaQjenxx0QtPTKNM...,4000~5000,1000~2000
2,%2BB7KaqyVcM77I5me2CAt7YAglTNGD5Dc5GH%2BnDDWdd...,3000~4000,1000~2000
3,%2BCOKHDKdkZqqKjForTGdnLm3MmDd8IU6p92T7WD%2FxJ...,1000~2000,1000~2000
4,%2BCSbH4dEHTU87B4l5wIL2sPR05x39YenLja%2BwhJHfC...,2000~3000,1000~2000
...,...,...,...
1957,zXbEjf7v8c3mEglJuvrOgAfDJLOKPfMIJBtWaaGbbmE%3D,1000~2000,0~1000
1958,zYfZEGjb87aleR0%2Bo%2FwppFEE0OUeB%2BBzelZRSgsR...,3000~4000,0~1000
1959,ZYovoMuuoGJaCyrO98oO1sc3y0xvMBddBPBSwmYOSSM%3D,3000~4000,3000~4000
1960,ZYoZybivohs6CXXKo6DxKeh4%2FLRU%2BqJT3Gluy1QtDd...,2000~3000,2000~3000


## 處理 non-numerical data

In [11]:
data7['人群標籤'] = [0] * len(data7)
data8['人群標籤'] = [1] * len(data8)

In [12]:
# 去除重複 uid
data7 = data7.drop_duplicates(subset='MemberID', keep="first")
data8 = data8.drop_duplicates(subset='MemberID', keep="first")
print(len(data7), len(data8))

1569 1958


In [13]:
remain_uid = list(direct.loc[(direct['APP uid'].isin(list(data7['MemberID'])))]['行為資料 uid'].unique())
len(remain_uid)

1569

In [14]:
uids = []

for uid in list(data7['MemberID']):
    uids.append(direct.loc[direct['APP uid'] == uid]['行為資料 uid'].iloc[0])

In [15]:
data7['uid'] = uids

In [16]:
uids = []

for uid in list(data8['MemberID']):
    uids.append(ad.loc[ad['APP uid'] == uid]['行為資料 uid'].iloc[0])

In [17]:
data8['uid'] = uids

In [18]:
data1 = data1.loc[data1['uid'].isin(remain_uid)]
data2 = data2.loc[data2['uid'].isin(list(ad['行為資料 uid'].unique()))]
data3 = data3.loc[data3['uid'].isin(remain_uid)]
data4 = data4.loc[data4['uid'].isin(list(ad['行為資料 uid'].unique()))]

In [19]:
print(len(data1), len(data2), len(data3), len(data4))

1569 1958 1569 1958


In [20]:
merge_direct = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, on=['uid'], 
                                              how='outer'), [data7, data1]).fillna('none')

In [21]:
merge_ad = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, on=['uid'], 
                                              how='outer'), [data8, data2]).fillna('none')

In [22]:
merge_direct = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, on=['uid'], 
                                              how='outer'), [merge_direct, data3]).fillna('none')
merge_ad = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, on=['uid'], 
                                              how='outer'), [merge_ad, data4]).fillna('none')

In [23]:
member_data = pd.concat([merge_direct, merge_ad])

In [24]:
member_data.columns

Index(['MemberID', 'Age', 'Gender', 'Birthday', 'IsEnableEmail',
       'IsEnablePushNotification', 'IsEnableShortMessage', 'MemberCardLevel',
       'platform', 'os', 'browser', 'location_original', 'location',
       'location_highest', 'habit_weekday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday', 'habit_hour', '0-6', '6-12',
       '12-18', '18-24', 'month_usage_score', '人群標籤', 'uid', 'viewproduct_all',
       'add_all', 'checkout_all', 'ave_purchase_grade', '累計金額', '每次平均消費金額'],
      dtype='object')

In [25]:
member_data.columns

Index(['MemberID', 'Age', 'Gender', 'Birthday', 'IsEnableEmail',
       'IsEnablePushNotification', 'IsEnableShortMessage', 'MemberCardLevel',
       'platform', 'os', 'browser', 'location_original', 'location',
       'location_highest', 'habit_weekday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday', 'Friday', 'Saturday', 'Sunday', 'habit_hour', '0-6', '6-12',
       '12-18', '18-24', 'month_usage_score', '人群標籤', 'uid', 'viewproduct_all',
       'add_all', 'checkout_all', 'ave_purchase_grade', '累計金額', '每次平均消費金額'],
      dtype='object')

In [26]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
member_data['Gender_process'] = lb_make.fit_transform(member_data['Gender'].astype(str))
member_data['IsEnableEmail_process'] = lb_make.fit_transform(member_data['IsEnableEmail'].astype(str))
member_data['IsEnablePushNotification_process'] = lb_make.fit_transform(member_data['IsEnablePushNotification'].astype(str))
member_data['IsEnableShortMessage_process'] = lb_make.fit_transform(member_data['IsEnableShortMessage'].astype(str))
member_data['platform_process'] = lb_make.fit_transform(member_data['platform'].astype(str))
member_data['os_process'] = lb_make.fit_transform(member_data['os'].astype(str))
member_data['browser_process'] = lb_make.fit_transform(member_data['browser'].astype(str))
member_data['location_highest_process'] = lb_make.fit_transform(member_data['location_highest'].astype(str))
member_data['cumulated_value'] = lb_make.fit_transform(member_data['累計金額'].astype(str))
member_data['average_cumulated_value'] = lb_make.fit_transform(member_data['每次平均消費金額'].astype(str))

In [27]:
member_data[['location_highest','location_highest_process']]

,location_highest,location_highest_process
0,台中,1
1,台北,2
2,台北,2
3,台北,2
4,高雄,35
...,...,...
1953,新北,24
1954,台北,2
1955,新北,24
1956,高雄,35


## 建立購買意圖分數標籤

In [28]:
score_label = []

for score in list(member_data['ave_purchase_grade']):
    if score < 10:
        score_label.append(0)
    elif score >= 10 and score < 20:
        score_label.append(1)
    elif score >= 20 and score < 40:
        score_label.append(2)
    elif score >= 40 and score < 60:
        score_label.append(3)
    else:
        score_label.append(5)

member_data['購買意圖標籤'] = score_label

In [29]:
member_data['購買意圖標籤']

0       5
1       5
2       0
3       3
4       0
       ..
1953    5
1954    3
1955    2
1956    0
1957    5
Name: 購買意圖標籤, Length: 3527, dtype: int64

In [30]:
member_data.to_csv('會員資料和購買意圖整理對照表.csv', index = False, encoding='big5')

In [32]:
member_data = pd.read_csv('會員資料和購買意圖整理對照表.csv', encoding='big5')

In [33]:
score_label = []

for score in list(member_data['ave_purchase_grade']):
    if score < 20:
        score_label.append(0)
    elif score >= 20 and score < 60:
        score_label.append(1)
    else:
        score_label.append(2)

member_data['購買意圖標籤_new'] = score_label

In [34]:
member_data.to_csv('會員資料和購買意圖整理對照表_with_new_score_label.csv', index = False, encoding='big5')

## 依照兩群人分開檔案

In [35]:
direct_data = member_data.loc[member_data['人群標籤'] == 0]

In [36]:
ad_data = member_data.loc[member_data['人群標籤'] == 1]

In [37]:
direct_data.to_csv('會員資料+購買意圖整理對照表_直接購買.csv', index = False, encoding='big5')
ad_data.to_csv('會員資料+購買意圖整理對照表_影響最深.csv', index = False, encoding='big5')

## 整理影響最深資料

In [49]:
ad = pd.read_csv('training data/會員資料+購買意圖整理對照表_影響最深.csv', encoding='big5')

In [50]:
ad['uid']

0        qikOnApfO8fGPvmETsk2UdQtquDmj6PdGli%2BflLA5vU%3D
1       06vUSRrOovhRI%2FikR2Rn7U0%2BrEcAGnuYXP0kcaufy2...
2          wjr0DprHIp91CWjCEec7a3urwaBrUeWG5g9amlbuKlg%3D
3        4klVm1xIdWF0vkBXz6FhMzo7%2F7ngkScb3HsHeziDSrM%3D
4          rudBuDaPgcgix8NT879P7HSiQFiJyZp4qEafwxExMg4%3D
                              ...                        
1953       0y36Ex0BjVoz2E5L9NR6XrSZle0VeNVnnpHdQlIA9z0%3D
1954       JsJXGZ7IarY3DbE2pW4t1BIvW4itykD6PzW2ocFHywg%3D
1955    y%2B4YTkK79%2F%2FfX2kG8TUqvYx2otO9D6OLW%2B2du%...
1956    5I6ubhY%2BJwFcgfC8i3%2FiS4iQXQ0PWMroVp%2FNIpxE...
1957    %2FMIQHuLB7mLpwR%2FekuJEnlxw0EgBD1oDRgk24QcuDr...
Name: uid, Length: 1958, dtype: object

In [51]:
data5 = data5.loc[data5['uid'].isin(list(ad['uid'].unique()))]
data6 = data6.loc[data6['uid'].isin(list(ad['uid'].unique()))]

In [52]:
# 去除重複 uid
data5 = data5.drop_duplicates(subset='uid', keep="first")
data6 = data6.drop_duplicates(subset='uid', keep="first")
print(len(data5), len(data6))

1772 1706


In [53]:
data5

,uid,cs_v1,cs_v2
0,qikOnApfO8fGPvmETsk2UdQtquDmj6PdGli%2BflLA5vU%3D,"['FB:0.95', '塔圖科技:0.03', 'IG:0.02']",FB
1,06vUSRrOovhRI%2FikR2Rn7U0%2BrEcAGnuYXP0kcaufy2...,"['Google:0.35', 'FB:0.23', '塔圖科技:0.04', 'Youtu...",Youtube
2,wjr0DprHIp91CWjCEec7a3urwaBrUeWG5g9amlbuKlg%3D,"['Google:0.07', 'FB:0.93']",FB
3,4klVm1xIdWF0vkBXz6FhMzo7%2F7ngkScb3HsHeziDSrM%3D,"['Google:0.89', 'FB:0.11']",Google
4,rudBuDaPgcgix8NT879P7HSiQFiJyZp4qEafwxExMg4%3D,"['FB:0.77', '塔圖科技:0.16', '宇匯知識科技:0.03', 'Googl...",FB
...,...,...,...
1776,0y36Ex0BjVoz2E5L9NR6XrSZle0VeNVnnpHdQlIA9z0%3D,"['FB:0.87', 'IG:0.01', 'Youtube:0.04', '宇匯知識科技...",FB
1777,JsJXGZ7IarY3DbE2pW4t1BIvW4itykD6PzW2ocFHywg%3D,['Google:1.0'],Google
1778,y%2B4YTkK79%2F%2FfX2kG8TUqvYx2otO9D6OLW%2B2du%...,['FB:1.0'],FB
1779,5I6ubhY%2BJwFcgfC8i3%2FiS4iQXQ0PWMroVp%2FNIpxE...,['Google:1.0'],Google


In [54]:
merge_ad = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, on=['uid'], 
                                              how='outer'), [ad, data5]).fillna('none')

In [55]:
merge_ad = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, on=['uid'], 
                                              how='outer'), [merge_ad, data6]).fillna('none')

In [56]:
merge_ad['cs_process'] = lb_make.fit_transform(merge_ad['cs_v2'].astype(str))
merge_ad['cm_process'] = lb_make.fit_transform(merge_ad['cm_v2'].astype(str))

In [59]:
merge_ad.to_csv('training data/影響最深-所有資料對照表.csv', index=False, encoding='big5')

In [60]:
merge_ad

,MemberID,Age,Gender,Birthday,IsEnableEmail,IsEnablePushNotification,IsEnableShortMessage,MemberCardLevel,platform,os,...,cumulated_value,average_cumulated_value,購買意圖標籤,購買意圖標籤_new,cs_v1,cs_v2,cm_v1,cm_v2,cs_process,cm_process
0,qikOnApfO8fGPvmETsk2UdQtquDmj6PdGli+flLA5vU=,36,Female,1984/11/22,True,True,True,20,[PC],[Windows],...,1,1,3,1,"['FB:0.95', '塔圖科技:0.03', 'IG:0.02']",FB,"['referral:0.23', '其他:0.72', 'cpc:0.03', 'FB:0...",其他,2,12
1,06vUSRrOovhRI/ikR2Rn7U0+rEcAGnuYXP0kcaufy24=,121,none,1900/1/1,True,True,True,30,[PC],[Windows],...,31,1,2,1,"['Google:0.35', 'FB:0.23', '塔圖科技:0.04', 'Youtu...",Youtube,"['cpc:0.38', '其他:0.46', 'referral:0.15']",其他,9,12
2,wjr0DprHIp91CWjCEec7a3urwaBrUeWG5g9amlbuKlg=,49,Female,1971/8/30,True,True,True,20,[PC],[Windows],...,24,1,1,0,"['Google:0.07', 'FB:0.93']",FB,"['cpc:0.07', '其他:0.83', 'referral:0.1']",其他,2,12
3,4klVm1xIdWF0vkBXz6FhMzo7/7ngkScb3HsHeziDSrM=,28,Female,1993/1/16,False,True,True,10,[PC],[Windows],...,31,1,2,1,"['Google:0.89', 'FB:0.11']",Google,"['cpc:0.42', '其他:0.11', 'organic:0.47']",organic,3,9
4,rudBuDaPgcgix8NT879P7HSiQFiJyZp4qEafwxExMg4=,121,none,1900/1/1,True,True,True,10,[PC],[Windows],...,2,0,1,0,"['FB:0.77', '塔圖科技:0.16', '宇匯知識科技:0.03', 'Googl...",FB,"['其他:0.49', 'cpc:0.19', 'referral:0.33']",其他,2,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1953,0y36Ex0BjVoz2E5L9NR6XrSZle0VeNVnnpHdQlIA9z0=,39,Female,1981/9/27,False,True,True,10,[Mobile],[Android],...,2,1,5,2,"['FB:0.87', 'IG:0.01', 'Youtube:0.04', '宇匯知識科技...",FB,"['其他:0.83', 'referral:0.16', 'StoreList:0.02']",其他,2,12
1954,JsJXGZ7IarY3DbE2pW4t1BIvW4itykD6PzW2ocFHywg=,121,none,1900/1/1,True,True,True,10,[PC],[Windows],...,24,1,3,1,['Google:1.0'],Google,['cpc:1.0'],cpc,3,5
1955,y+4YTkK79//fX2kG8TUqvYx2otO9D6OLW+2du/FkUW0=,39,Female,1981/10/31,True,True,True,20,[Mobile],[Android],...,2,1,2,1,['FB:1.0'],FB,"['其他:0.83', 'StoreList:0.17']",其他,2,12
1956,5I6ubhY+JwFcgfC8i3/iS4iQXQ0PWMroVp/NIpxE51k=,35,Female,1985/6/16,False,True,True,10,[PC],[Others],...,2,1,0,0,['Google:1.0'],Google,['cpc:1.0'],cpc,3,5
